In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np


In [15]:
results = {}

In [35]:
def lstm_predict(train_X, train_y, test_X):
    """ lstm_predict """
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(10, 1))) # 50 LSTM units
    model.add(Dense(1)) # Output layer
    # Compile the model
    model.compile(optimizer='adam', loss='mse')
    # Train the model
    model.fit(train_X, train_y, epochs=100, verbose=0)
    # Make predictions
    X_test = np.random.rand(5, 10, 1)
    predictions = model.predict(test_X)
    return predictions

In [36]:
def load_metrics(random_state, version, db_file_path):
    query = f'select rounds, matched_cnts from rndforest where random_state={random_state} and version=\\"{version}\\"'
    metrics = !echo {query} | sqlite3 {db_file_path}
    metrics = metrics[0].split('|')
    return metrics[0][0], metrics[1]


version = "T_01_10"
db_file_path = '../db/metrics.db'
random_state = 113789
rounds, matched_cnts = load_metrics(random_state=random_state, db_file_path=db_file_path, version=version)
print(metrics)
train_X = []
train_y = []
test_X = 1376
for i in rounds.split(','):
    train_X.append(int(i))
for i in matched_cnts.split(','):
    train_y.append(int(i))
predict_num = lstm_predict(train_X=train_X, train_y=train_y, test_X=test_X)
print(f'predicted = {predicted_num}')

['1175,1174,1173,1172,1171,1170,1169,1168,1167,1166,1165,1164,1163,1162,1161,1160,1159,1158,1157,1156,1155,1154,1153,1152,1151,1150,1149,1148,1147,1146,1145,1144,1143,1142,1141,1140,1139,1138,1137,1136,1135,1134,1133,1132,1131,1130,1129,1128,1127,1126,1125,1124,1123,1122,1121,1120,1119,1118,1117,1116,1115,1114,1113,1112,1111,1110,1109,1108,1107,1106,1105,1104,1103,1102,1101,1100,1099,1098,1097,1096,1095,1094,1093,1092,1091,1090,1089,1088,1087,1086,1085,1084,1083,1082,1081,1080,1079,1078,1077,1076,1075,1074,1073,1072,1071,1070,1069,1068,1067,1066,1065,1064,1063,1062,1061,1060,1059,1058,1057,1056,1055,1054,1053,1052,1051,1050,1049,1048,1047,1046,1045,1044,1043,1042,1041,1040,1039,1038,1037,1036,1035,1034,1033,1032,1031,1030,1029,1028,1027,1026,1025,1024,1023,1022,1021,1020,1019,1018,1017,1016,1015,1014,1013,1012,1011,1010,1009,1008,1007,1006,1005,1004,1003,1002,1001,1000,999,998,997,996,995,994,993,992,991,990,989,988,987,986,985,984,983,982,981,980,979,978,977,976,975,974,973,972,971,97

/opt/conda/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Unrecognized data type: x=[1] (of type <class 'list'>)

In [ ]:
import pandas as pd
import numpy as np
import requests
import io
from tqdm import tqdm

# 딥러닝 라이브러리 TensorFlow/Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MultiLabelBinarizer

# 시퀀스 길이 (과거 몇 회차의 데이터를 보고 다음을 예측할 것인가)
SEQUENCE_LENGTH = 10


def create_sequences(data, seq_length):
    """
    데이터를 LSTM 학습에 맞는 시퀀스 형태로 변환합니다.
    """
    xs = []
    ys = []
    # Multi-hot 인코딩을 위해 1~45번까지의 클래스를 명시적으로 지정
    mlb = MultiLabelBinarizer(classes=list(range(1, 46)))
    
    # 전체 데이터를 Multi-hot 벡터로 변환
    encoded_data = mlb.fit_transform(data)
    
    for i in tqdm(range(len(encoded_data) - seq_length), desc="시퀀스 데이터 생성 중"):
        # 입력 시퀀스 (X): i부터 i+seq_length-1 까지
        xs.append(encoded_data[i:(i + seq_length)])
        # 타겟 (y): i+seq_length
        ys.append(encoded_data[i + seq_length])
        
    return np.array(xs), np.array(ys), mlb


def build_lstm_model(seq_length, num_features, neural_num=128, verbose=0, activation='sigmoid'):
    """
    LSTM 모델을 생성합니다.
    """
    model = Sequential()
    # 입력층: LSTM 레이어, input_shape=(시퀀스 길이, 피처 개수)
    model.add(LSTM(neural_num, return_sequences=True, activation=activation, input_shape=(seq_length, num_features)))
    model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    # model.add(LSTM(neural_num, return_sequences=True, activation=activation))
    model.add(Dropout(0.2))
    # 중간층: LSTM 레이어
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.2))
    # 출력층으로 가기 전 Dense 레이어
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_features, activation='sigmoid'))
    
    # 모델 컴파일
    # binary_crossentropy는 각 번호가 나올지/안나올지를 독립적으로 판단하는 다중 라벨 문제에 적합
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    if verbose > 0:
        model.summary()
    return model


file_names = [
    (1174, 'xlsx/1173.xlsx', [8, 11, 14, 17, 36, 39]),
    (1175, 'xlsx/1174.xlsx', [3, 4, 6, 8, 32, 42]),
    (1176, 'xlsx/1175.xlsx', [0, 0, 0, 0, 0, 0])
    ]



for file_name in file_names:
    # --- 코드 실행 ---
    # 1. 역대 당첨 번호 데이터 가져오기
    lotto_history = get_lotto_history(*file_name, row_end=10)
    if lotto_history is not None:
        neural_num = 64
        # 2. 데이터 전처리 및 시퀀스 생성
        X, y, mlb = create_sequences(lotto_history, SEQUENCE_LENGTH)
    
        # 3. LSTM 모델 빌드
        # num_features는 로또 번호의 개수(45)
        model = build_lstm_model(SEQUENCE_LENGTH, X.shape[2], neural_num=neural_num)
    
        # 4. 모델 학습 (데이터가 많아 시간이 소요될 수 있습니다)
        print("\nLSTM 모델 학습을 시작합니다...")
        model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1, verbose=0)
    
        # 5. 1176회 예측
        print(f"\{file_name[0]}회 예측을 시작합니다...")
    
        # 예측을 위해 가장 마지막 시퀀스를 입력으로 사용
        last_sequence_raw = lotto_history[-SEQUENCE_LENGTH:]
        last_sequence_encoded = mlb.transform(last_sequence_raw)
    
        # Keras 모델은 배치 입력을 기대하므로 차원을 추가 (samples, timesteps, features)
        input_for_prediction = np.expand_dims(last_sequence_encoded, axis=0)
    
        # 예측 실행 (결과는 각 번호가 나올 확률)
        predicted_probabilities = model.predict(input_for_prediction)[0]
    
        # 확률이 높은 순으로 6개 번호의 인덱스를 추출
        # argsort는 값을 정렬했을 때의 원래 인덱스를 반환
        top_6_indices = np.argsort(predicted_probabilities)[-6:]
    
        # 인덱스는 0부터 시작하므로 1을 더해 실제 로또 번호로 변환
        predicted_numbers = sorted([int(i + 1) for i in top_6_indices])
        if file_name[0] not in results:
            results[file_name[0]] = [predicted_numbers]
        else:
            results[file_name[0]].append(predicted_numbers)
    
        print("\n--- 최종 예측 결과 ---")
        print(f"LSTM 모델 기반 {file_name[0]}회 예측 번호: {predicted_numbers} {neural_num}, ")

In [13]:
version = "T_01_10"
db_file_path = '../db/metrics.db'
random_state = 113789
metrics = load_metrics(random_state=random_state, db_file_path=db_file_path, version=version)
print(metrics)

query select rounds, matched_cnts from rndforest where random_state=113789 and version="T_01_10";
db_file_path ../db/metrics.db
["'select rounds, matched_cnts from rndforest where random_state=113789 and version=T_01_10", "/bin/bash: line 1: ': command not found"]


In [41]:
for result in results:
    print("라운드", result)
    for metric in results[result]:
        print(metric)

라운드 1174
[12, 18, 21, 30, 33, 45]
[12, 18, 21, 33, 34, 38]
[12, 15, 18, 21, 33, 34]
[6, 7, 33, 34, 38, 45]
[12, 13, 15, 33, 34, 45]
[11, 12, 21, 33, 38, 45]
[12, 18, 21, 33, 34, 38]
[12, 18, 33, 34, 38, 45]
[12, 21, 28, 33, 38, 45]
[12, 15, 18, 33, 34, 38]
[12, 15, 33, 34, 38, 45]
라운드 1175
[6, 12, 21, 33, 34, 38]
[11, 12, 21, 33, 38, 45]
[12, 19, 21, 33, 38, 45]
[7, 11, 12, 18, 33, 34]
[12, 18, 21, 33, 38, 45]
[6, 11, 12, 33, 34, 45]
[12, 19, 21, 33, 38, 45]
[3, 12, 18, 21, 33, 38]
[12, 15, 18, 33, 38, 45]
[3, 12, 13, 21, 33, 38]
[12, 18, 21, 33, 34, 38]
라운드 1176
[3, 6, 12, 21, 33, 38]
[6, 12, 15, 21, 33, 38]
[7, 11, 12, 21, 38, 45]
[12, 13, 33, 34, 38, 45]
[6, 12, 33, 34, 38, 45]
[12, 18, 21, 33, 38, 45]
[12, 13, 33, 34, 38, 45]
[12, 19, 21, 33, 34, 45]
[12, 13, 15, 21, 33, 45]
[12, 21, 33, 34, 38, 45]
[12, 18, 21, 34, 38, 45]


In [7]:

actual_numbers = [
    [1175, 3, 4, 6, 8, 32, 42],
    [1174, 8, 11, 14, 17, 36, 39],
    [1173, 1, 5, 18, 20, 30, 35],
    [1172, 7, 9, 24, 40, 42, 44],
    [1171, 3, 6, 7, 11, 12, 17],
    [1170, 3, 13, 28, 34, 38, 42],
    [1169, 5, 12, 24, 26, 39, 42],
    [1168, 9, 21, 24, 30, 33, 37],
    [1167, 8, 23, 31, 35, 39, 40],
]